In [1]:
import pandas as pd
import numpy as np

# Create array_dict from an array in a .csv file

## Read the file with protection from oddball encoders.

In [2]:
def safe_csv_read(filename, usecols=None, index_col=None):
    ## Read the file with protection from oddball encoders.
    try:
        return pd.read_csv(filename, usecols=usecols, index_col=index_col)
    except UnicodeDecodeError:
        return pd.read_csv(filename, usecols=usecols, index_col=index_col, encoding="ISO-8859-1")


## *Load* ***array_dict*** *using* ***safe_csv_read()*** *set index,convert data list to np.arrays*

In [3]:
def load_group_arrays(filename: str, group_name: str, grp_list: list[str], index_col: str = "dtv") -> dict[str, np.ndarray]:
    ## *Load the array using* ***safe_csv_read()*** *set index,convert data list to np.arrays & build* ***array_dict***
    
    columns_to_load = [index_col] + grp_list
    df = safe_csv_read(filename, usecols=columns_to_load)
    # before safe_csv_read df = pd.read_csv(filename, usecols=columns_to_load, encoding=encoding)
    df.set_index(index_col, inplace=True)

    array_dict = {f"{group_name}_{col}": df[col].to_numpy() for col in grp_list}
    array_dict[f"{group_name}_{index_col}"] = df.index.to_numpy()  # include index as array
    return array_dict

## *Set* ***filename, group_name, grp_list*** then *call* ***load_group_arrays()*** *to compute np.arrays & build* ***array_dict***

In [4]:
## *Set* ***filename, group_name, grp_list*** then *call* ***load_group_arrays()*** *to compute np.arrays & build* ***array_dict***
filename = "xl_dtv_all_snm_tst.csv"
group_name = "grp_tst"
grp_list =["Eliquist","Magtien"]
# before group array   array_dict = load_group_arrays(filename, group_name , grp_list, index_col = "DATE_V", encoding = "ISO-8859-1")
array_dict = load_group_arrays(filename, group_name , grp_list, index_col = "dtv")

In [5]:
# checkout
array_dict

{'grp_tst_Eliquist': array([10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1., nan, nan]),
 'grp_tst_Magtien': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., nan, nan]),
 'grp_tst_dtv': array([45849, 45850, 45851, 45852, 45853, 45854, 45855, 45856, 45857,
        45858, 45859, 45860])}

In [6]:
# checkout
array_dict['grp_tst_dtv'][5]

np.int64(45854)

In [7]:
# checkout
array_dict['grp_tst_Magtien'][9]

np.float64(10.0)

## *From a ***dtv*** find* ***row_index*** *for a ***specific dtv*** *then use* ***row_index()*** *to find the value in a given dat_col np.array to get the* ***datcol_value*** for that ***specific dtv***

In [ ]:
# Load the array using safe_csv_read() 
# set index,convert data list to np.arrays & build array_dict
dtv_array = array_dict["grp_tst_dtv"]
grp_tst_dtv = 45852

# Find the index position
row_index = np.where(dtv_array == grp_tst_dtv)[0]

if row_index.size > 0:
    print(f"✅ row_index for {grp_tst_dtv}: {row_index[0]}")
else:
    print(f"⚠️ Date {grp_tst_dtv} not found in 'grp_tst_dtv'")
# print("row_index  =  ",row_index)

In [ ]:
array_dict['grp_tst_Magtien'][row_index]

# Manual Input to array_dict

## BASIC SIMPLWE INPUT FUNCTION *** does not save entries yet*** Interactive Data Entry Function [not addapted from copilot]

In [8]:
array_dict           # non essentual readout

{'grp_tst_Eliquist': array([10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1., nan, nan]),
 'grp_tst_Magtien': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., nan, nan]),
 'grp_tst_dtv': array([45849, 45850, 45851, 45852, 45853, 45854, 45855, 45856, 45857,
        45858, 45859, 45860])}

In [9]:
def enter_data_values_dtv(array_dict: dict, group_name: str, col_list: list[str], index_key: str = "dtv") -> None:
    #     # def to load values of doses for a given "dtv"
    # Get master index
    master_index = array_dict.get(f"{group_name}_{index_key}")
    if master_index is None:
        raise KeyError(f"❌ Master index '{group_name}_{index_key}' not found in array_dict.")

    # Loop until valid dtv or abort
    while True:
        try:
            dtv_input = int(input("📅 Enter the date (format: NNNNN): ").strip())
        except ValueError:
            print("❌ Invalid format. Please enter a numeric date.")
            continue

        if dtv_input in master_index:
            break  # valid date, proceed
        else:
            choice = input(f"⚠️ '{dtv_input}' not found in timeline. Re-enter (r) or Abort (a)? ").strip().lower()
            if choice == "a":
                print("🚫 Entry aborted.")
                return

    # Collect values interactively
    new_values = {}
    for col in col_list:
        val = input(f"🔢 Enter value for '{col}': ").strip()
        try:
            new_values[col] = float(val)
        except ValueError:
            new_values[col] = np.nan  # fallback if input is invalid

    # Create new DataFrame with dtv index
    df_new = pd.DataFrame(new_values, index=[dtv_input])
    df_new.index.name = index_key

    # Reindex to match master timeline
    df_new = df_new.reindex(master_index)

    # Append each column to array_dict
    for col in col_list:
        key = f"{group_name}_{col}"
        array_dict[key] = df_new[col].to_numpy()
        print(f"✅ Updated '{key}' with new entry for {dtv_input}")


## CLASSES AND SKIPS ADDED***  does not save entries yet*** Interactive Data Entry Function   [not INTEGRATED  from copilot]

### ###########Question to copipol###################
Yesterday you help build the follow in input function and it is working fine. ======================================= def enter_data_values_dtv(array_dict: dict, group_name: str, col_list: list[str], index_key: str = "dtv") -> None: # # def to load values of doses for a given "dtv" # Get master index master_index = array_dict.get(f"{group_name}{index_key}") if master_index is None: raise KeyError(f"❌ Master index '{group_name}{index_key}' not found in array_dict.") # Loop until valid dtv or abort while True: try: dtv_input = int(input("📅 Enter the date (format: NNNNN): ").strip()) except ValueError: print("❌ Invalid format. Please enter a numeric date.") continue if dtv_input in master_index: break # valid date, proceed else: choice = input(f"⚠️ '{dtv_input}' not found in timeline. Re-enter (r) or Abort (a)? ").strip().lower() if choice == "a": print("🚫 Entry aborted.") return # Collect values interactively new_values = {} for col in col_list: val = input(f"🔢 Enter value for '{col}': ").strip() try: new_values[col] = float(val) except ValueError: new_values[col] = np.nan # fallback if input is invalid # Create new DataFrame with dtv index df_new = pd.DataFrame(new_values, index=[dtv_input]) df_new.index.name = index_key # Reindex to match master timeline df_new = df_new.reindex(master_index) # Append each column to array_dict for col in col_list: key = f"{group_name}_{col}" array_dict[key] = df_new[col].to_numpy() print(f"✅ Updated '{key}' with new entry for {dtv_input}") ==========call ============================ group_name = "grp_tst" col_list = ["Eliquist", "Magtien", "GPLC"] enter_data_values_dtv(array_dict, group_name, col_list) ============================================== I would like modify our this function to include classes . These are the following steps The input function that accepts a "dtv" , sets the row in a data_dict and enters enter value in each dat_col. [unchanged] adding classes ------------------ Each "dat_col" belongs to a "supplement class" attribute = name[3bits] attribute = mrn_dose[3bits] attribute = default_mrn_dose[3bits] attribute = value[list for all rows] filled_flag = [1bit] active_flag =[1bit] the row = dvt[integer] the function will loop through ) if the "(dat_col.active_flag " = false or the ": "dat_col.filled_flag " = true. it moves on to the next "dat_col when finding a col that meets the conditions for entry------------- it displays the :dat_col.name & dat_col,mrn_dose & dat_col.default_mrn_dose & the "dtv" If I hit "enter" , I set "dat_col,mrn_dose.value[dtv] is set to dat_col.default_mrn_dose then it sets " "dat_col.filled_flag " = true, and moves on to next "dat_col" If I hit the "right arrow" it just sets the "dat_col.filled_flag " = true, and moves on to next "dat_col" Until it completes all dat_col.


### 1. Define the Supplement Class

In [ ]:
class Supplement:
    def __init__(self, name: str, default_mrn_dose: float, timeline: list[int]):
        self.name = name[:3]  # 3-bit name
        self.mrn_dose = [np.nan] * len(timeline)  # 3-bit dose list
        self.default_mrn_dose = default_mrn_dose  # 3-bit default
        self.value = [np.nan] * len(timeline)  # full timeline values
        self.filled_flag = False  # 1-bit
        self.active_flag = True   # 1-bit
        self.timeline = timeline  # reference for indexing

    def set_value(self, dtv: int, val: float):
        try:
            idx = self.timeline.index(dtv)
            self.value[idx] = val
            self.mrn_dose[idx] = val
            self.filled_flag = True
        except ValueError:
            print(f"❌ DTV {dtv} not found in timeline.")


### 2. Refactor the Input Function

In [ ]:
def enter_supplement_values(supplements: list[Supplement], dtv: int) -> None:
    for supp in supplements:
        if not supp.active_flag or supp.filled_flag:
            continue

        print(f"\n🧾 Supplement: {supp.name}")
        print(f"💊 Default Dose: {supp.default_mrn_dose}")
        print(f"📅 Date: {dtv}")

        user_input = input("⏎ Enter to accept default, → to skip, or type dose: ").strip()

        if user_input == "":
            supp.set_value(dtv, supp.default_mrn_dose)
            print(f"✅ Set to default: {supp.default_mrn_dose}")
        elif user_input == "→":
            supp.filled_flag = True
            print("➡️ Skipped.")
        else:
            try:
                dose = float(user_input)
                supp.set_value(dtv, dose)
                print(f"✅ Set to entered dose: {dose}")
            except ValueError:
                print("❌ Invalid input. Skipping.")
                continue


### 3. Integrate with Timeline and Array Dict

In [ ]:
def run_supplement_entry(array_dict: dict, group_name: str, supplement_defs: dict[str, float], index_key: str = "dtv") -> None:
    master_index = array_dict.get(f"{group_name}{index_key}")
    if master_index is None:
        raise KeyError(f"❌ Master index '{group_name}{index_key}' not found in array_dict.")

    # Get valid DTV
    while True:
        try:
            dtv_input = int(input("📅 Enter the date (format: NNNNN): ").strip())
        except ValueError:
            print("❌ Invalid format. Please enter a numeric date.")
            continue

        if dtv_input in master_index:
            break
        else:
            choice = input(f"⚠️ '{dtv_input}' not found. Re-enter (r) or Abort (a)? ").strip().lower()
            if choice == "a":
                print("🚫 Entry aborted.")
                return

    # Create Supplement objects
    supplements = [Supplement(name, default, master_index) for name, default in supplement_defs.items()]

    # Run entry loop
    enter_supplement_values(supplements, dtv_input)

    # Update array_dict
    for supp in supplements:
        key = f"{group_name}_{supp.name}"
        array_dict[key] = supp.value
        print(f"📦 Stored '{key}' in array_dict.")


### ✅ Example Call

In [ ]:
supplement_defs = {
    "Eliquist": 5.0,
    "Magtien": 2.0,
    "GPLC": 3.5
}

run_supplement_entry(array_dict, "grp_tst", supplement_defs)


# SAVE DATA UPDATES FROM ***array_dict*** TO XLFILE 

In [11]:
array_dict            # non essentual readout

{'grp_tst_Eliquist': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 60.]),
 'grp_tst_Magtien': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 30.]),
 'grp_tst_dtv': array([45849, 45850, 45851, 45852, 45853, 45854, 45855, 45856, 45857,
        45858, 45859, 45860]),
 'grp_tst_GPLC': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 21.])}

## 45858 Add a new column [not addapted from copilot]

# Start of setting up classes

## -Oriented DataCObjectolumn Design in Python
https://copilot.microsoft.com/shares/pages/MZyaioewJp2wR4mSwfShP

In [ ]:
import pandas as pd
import numpy as np

class TimeSeriesGroup:
    def __init__(self, filename: str, group_name: str, columns: list[str], index_col: str = "DATE_V", encoding: str = "ISO-8859-1"):
        self.group_name = group_name
        self.index_col = index_col
        self.filename = filename
        self.encoding = encoding
        self.columns = columns
        self.array_dict = {}

        self._load_data()

    def _load_data(self):
        cols_to_load = [self.index_col] + self.columns
        df = pd.read_csv(self.filename, usecols=cols_to_load, encoding=self.encoding)
        df.set_index(self.index_col, inplace=True)

        # Store index as array
        self.array_dict[f"{self.group_name}_{self.index_col}"] = df.index.to_numpy()

        # Store each column as array
        for col in self.columns:
            arr = df[col].to_numpy()
            self.array_dict[f"{self.group_name}_{col}"] = arr

    def get_array(self, name: str) -> np.ndarray:
        return self.array_dict.get(name)

    def summary(self):
        print(f"📦 Group: {self.group_name}")
        for key, arr in self.array_dict.items():
            print(f"  {key}: shape={arr.shape}, dtype={arr.dtype}, nulls={np.isnan(arr).sum()}")


## Using doses in classis 

***Python Class for Supplement Dosage Modeling***
https://copilot.microsoft.com/shares/pages/TWp5or9jyx8bJtPj5cGfv

You can definitely model ***actual_dose*** and ***default_dose*** as ***3-bit integers (values 0–7)***, and multiply them by a gr_per_dose scalar to compute the effective dosage in grams. Python doesn’t have a native 3-bit dtype, but you can enforce the constraint and optimize storage using NumPy or bit masking.


In [ ]:
import numpy as np

class Supplement:
    def __init__(self, name, gr_per_dose, default_dose=1, actual_dose=None):
        self.name = name
        self.gr_per_dose = gr_per_dose  # grams per dose unit
        self.default_dose = self._validate_dose(default_dose)
        self.actual_dose = self._validate_dose(actual_dose) if actual_dose is not None else self.default_dose

    def _validate_dose(self, dose):
        # Enforce 3-bit constraint (0–7)
        if not (0 <= dose <= 7):
            raise ValueError(f"Dose must be between 0 and 7 (3-bit range). Got: {dose}")
        return np.uint8(dose)  # Efficient storage

    @property
    def effective_dose_grams(self):
        return self.actual_dose * self.gr_per_dose

    def __repr__(self):
        return (f"{self.name}: {self.actual_dose} dose units × {self.gr_per_dose} g/unit = "
                f"{self.effective_dose_grams:.2f} g total")


In [ ]:
# ✅ Example Usage
vitamin_c = Supplement(name="Vitamin C", gr_per_dose=0.5, default_dose=2)
print(vitamin_c)  # Output: Vitamin C: 2 dose units × 0.5 g/unit = 1.00 g total


# Discriptions of ***array_dict***

***array_dict*** Is a working dictionary where  dtv is the index and selected dat_cols which are np.arrays
1. dtv is a col of integers representing the number of days since jan 1 1900 and ends with today.
2. dtv is used as the index for all dat_cols
3. all dat_cols have cls attributes common to all the values in that dat-col
5. all dat_cols have cls methods that instructions to certain statistics of the dat_col values.
6. dat_col attributes and methods are read by a . behind dat_col ie dat_col.method or attribute. ***see*** *https://copilot.microsoft.com/shares/pages/MZyaioewJp2wR4mSwfShP* for examples.
7. all have same length as the ***dtv col*** and are stored in a ***xl csv file array*** with ***"dtv"*** as the first col and str headers as the first row.
8. ***array_dict*** use the ***dtv*** as index so that every row has a ***row_index***
9. The ***row_index*** for a ***specific dtv*** is computed and it is used to find the value of ***dat_col*** for that date.  
10. ***Time Alignment*** It ensures all arrays—whether supplements, motion, or biochemical—are synchronized to the same timeline. That’s essential for cross-correlation, lag analysis, and time-shifting.
11. Plotting & Visualization You’ll need dtv as the x-axis for any time-series plots. Without it, your arrays are just floating sequences.
12. Diagnostics & Gaps You can detect missing data, irregular sampling, or temporal drift by comparing ***"dtv"*** across groups.
13. Batch Operations If you’re merging arrays from different domains (e.g., Motion, Supplements, Sleep), having a shared **"dtv"*** lets you align them cleanly—even if some arrays are sparse.
.................array_dict[f"{group_name}_dtv] = df.index.to_numpy()
---------------------------------------------------------------------
{
    "grp_tst_dtv": np.array([...]),
    "grp_tst_Eliquist": np.array([...]),
    "grp_tst_Magtien": np.array([...]),
    "grp_tst_GPLC": np.array([...])
}
-------------------------------------------------------------------------------------------

***Let’s scaffold a clean, modular TimeSeriesGroup class that fits your architecture and handles:***

✅ Selective column loading from a master CSV

✅ Index alignment via DATE_V

✅ Dictionary-style access to NumPy arrays

✅ Metadata logging for diagnostics and traceability